In [1]:
import pandas as pd
from requests.packages import target

positive_edges = pd.read_csv('../../data/edgelist.txt', sep=',', header=None)
positive_edges.columns = ['source', 'target']
tfidf_embeddings = pd.read_csv('../../data/abstracts_tfidf_embeddings.csv')
tfidf_embeddings.head(2)

,3d,able,abstract,accuracy,accurate,achieve,achieves,action,adaptive,addition,...,view,vision,visual,way,web,word,words,work,world,paper_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
positive_edges_cosine_similarity_df = pd.DataFrame(columns=['source', 'target', 'cosine_similarity'])

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm


positive_edges_cosine_similarity_df['source'] = positive_edges['source']
positive_edges_cosine_similarity_df['target'] = positive_edges['target']

cosine_similarities = []
for idx, row in tqdm(positive_edges_cosine_similarity_df.iterrows(), total=positive_edges_cosine_similarity_df.shape[0]):
    source_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['source']].iloc[:, 1:-1].values
    target_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['target']].iloc[:, 1:-1].values
    if source_vec.size > 0 and target_vec.size > 0:
        sim = cosine_similarity(source_vec, target_vec)[0][0]
    else:
        sim = None
    cosine_similarities.append(sim)

positive_edges_cosine_similarity_df['cosine_similarity'] = cosine_similarities

positive_edges_cosine_similarity_df.to_csv('../../data/training/positive/positive_edges_cosine_similarity.csv', index=False)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


  0%|          | 0/1091955 [00:00<?, ?it/s]